In [4]:
import numpy as np

import unicodedata
import re
import json
import random

import nltk
from nltk.stem.porter import PorterStemmer	
stemmer = PorterStemmer()

from transformers import BertTokenizer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import warnings 
warnings.filterwarnings('ignore')

from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

import joblib

In [5]:
from DZ1_data import GetData

In [6]:
elrian_dialogues, leonard_dialogues = GetData()

In [8]:
# Преобразование входных и выходных данных в строки
corpus = []
for elrian_dialogue, leonard_dialogue in zip(elrian_dialogues, leonard_dialogues):
    sentence = ' '.join([elrian_dialogue, leonard_dialogue])
    corpus.append(sentence)

# Запись строк в файл
with open('./data/corpus.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(corpus))

# Загрузка данных из файла
data_corpus = LineSentence('./data/corpus.txt')

In [9]:
# Обучение Word2Vec модели
word2vec_model = Word2Vec(sentences=data_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Функция для получения вектора предложения
def sentence_vectorizer(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Векторизация вопросов и ответов
question_vectors = [sentence_vectorizer(sentence, word2vec_model) for sentence in data_corpus]

In [11]:
# Сохранение Word2Vec модели
word2vec_model.save("./DZ1model/word2vec/model")

# Сохранение векторов
joblib.dump(question_vectors, "./DZ1model/word2vec/question_vectors.joblib")

['./DZ1model/word2vec/question_vectors.joblib']